#**Física Computacional - Trabajo Final**

## Modelo de Ising 2D


Basicamente armé una clase de Ising 2D, adaptando el código que se mostró en clase. Los únicos cambios relevantes realizados fueron
* La utilización de un array de temperaturas en vez de puntos equiespaciados.
* Se agrego la opcion de poner un campo h (Existe una version alternativa de la clase donde este bien optimizada esa opcion, como en mis simulaciones trabaje a campo nulo no lo puse)

# Clase de ISING2D

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cupy as cp


#Posibles valores de cambio de productos: espín central por suma de vecinos
J_prods = cp.array([-8, -4, 0, 4, 8], dtype=cp.float32)

# Arreglos Auxiliares
''' Índices para los factores de Boltzmann de acuerdo al
valor P de la suma de los espines vecinos x el espín central
 tabla_de_cambios[P + 4]'''

tabla_de_cambios = cp.array([0, 0, 1, 0, 2, 0, 3, 0, 4], dtype=cp.int32)


def idx(i, j, L):
  return i * L + j

def mis_vecinos(i, j, L):
  arriba = idx((i - 1) % L, j, L)
  abajo = idx((i + 1) % L, j, L)
  izquierda = idx(i, (j - 1) % L, L)
  derecha = idx(i, (j + 1) % L, L)
  return [arriba, abajo, izquierda, derecha]


#Bueno ahora voy a crear una clase para simular el modelo de ising en 2D:
class Ising2D:
  def __init__(self,Temps, L, random_init=False, h=0):
    self.J = 1
    self.kb = 1
    self.L = L
    self.h = h
    self.T_inicial = 0
    self.num_spin = L*L
    self.NTemp = len(Temps)
    self.Temps = Temps
    self.vecinos_blancos = []   #Armo un listado de vecinos
    self.vecinos_negros = []
    self.energia_vals = cp.zeros((self.NTemp, steps_mc))
    self.mag_vals = cp.zeros((self.NTemp, steps_mc))
    self.conf_inicial(random_init)
    #Aca tendria que agregar el self.conf_inicial()

    self.energia = self.energia_total()
    self.magnetizacion = self.magnetizacion_total()

  def conf_inicial(self, random_init):
    if random_init:
      self.espines_blancos = cp.random.choice(cp.array([-1, 1], dtype=cp.int32), size=self.num_spin // 2)
      self.espines_negros = cp.random.choice(cp.array([-1, 1], dtype=cp.int32), size=self.num_spin // 2)    #Condicion inicial aleatoria
    else:
      # Valores iniciales de los espines
      self.espines_blancos = cp.ones(self.num_spin // 2, dtype=cp.int32)
      self.espines_negros = cp.ones(self.num_spin // 2, dtype=cp.int32)                #Condicion inicial ferromagnética

    #Esta es la parte adaptada del arreglo de vecinos
    indice_blancos = 0
    indice_negros = 0
    for i in range(self.L):
      for j in range(self.L):
        self.vecinos = mis_vecinos(i, j, self.L)
        if (i + j) % 2 == 0:
            self.vecinos_blancos.append([(indice_blancos, n // 2) for n in self.vecinos])
            indice_blancos += 1
        else:
            self.vecinos_negros.append([(indice_negros, n // 2) for n in self.vecinos])
            indice_negros += 1
    self.vecinos_blancos = cp.array(self.vecinos_blancos, dtype=cp.int32)
    self.vecinos_negros = cp.array(self.vecinos_negros, dtype=cp.int32)


######
#####
  def simu(self, steps_mc, Neq):
    i = 0
    for k in self.Temps:
      T = k
      # Exponenciales para Metropolis
      exp_vals = cp.exp(-self.J * J_prods / T)

      for j in range(0,steps_mc):
          # Proponemos cambios en los espines "blancos" de manera secuencial.
          # Numeros uniforme [0,1] para decidir los cambios
          random_numbers = cp.random.rand(self.num_spin // 2)
          # Todos los índices de los vecinos de los espines "blancos"
          indices_vecinos = self.vecinos_blancos[:, :, 1]

          # Productos de espin central y suma de espines vecinos
          productos = self.espines_blancos * cp.sum(self.espines_negros[indices_vecinos], axis=1)

          # indices para las exponenciales
          indices = tabla_de_cambios[productos + 4]
          # Para cuales aceptamos cambio?
          to_flip = random_numbers < exp_vals[indices]
          # Invertimos los cambios aceptados
          self.espines_blancos[to_flip] *= -1

          #Actualiza
          self.magnetizacion += 2 * cp.sum(self.espines_blancos[to_flip])
          self.energia += 2 * self.J * cp.sum(productos[to_flip]) - self.h * cp.sum(self.espines_blancos[to_flip])

          # Lo mismo para los espines en los cuadros Negros
          random_numbers = cp.random.rand(self.num_spin // 2)
          indices_vecinos = self.vecinos_negros[:, :, 1]
          productos = self.espines_negros * cp.sum(self.espines_blancos[indices_vecinos], axis=1)
          indices = tabla_de_cambios[productos + 4]
          to_flip = random_numbers < exp_vals[indices]

          self.espines_negros[to_flip] *= -1
          self.magnetizacion += 2 * cp.sum(self.espines_negros[to_flip])
          self.energia += 2 * self.J * cp.sum(productos[to_flip]) - self.h * cp.sum(self.espines_negros[to_flip])

          self.energia_vals[i,j] = self.energia
          self.mag_vals[i,j] = self.magnetizacion
      print(f'Temp = {T}')
      i+=1

    if self.NTemp >1 :    #De momento lo pongo para no generar al pedo cuando tengo 1 sola temp.

      temperaturas = cp.array(self.Temps)
          # Calcular cantidades térmicas de manera vectorizada
      energia_promedio = cp.mean(self.energia_vals[:, Neq:], axis=1)
      energia2_promedio = cp.mean(self.energia_vals[:, Neq:]**2, axis=1)
      magnet_promedio = cp.mean(self.mag_vals[:, Neq:], axis=1)
      magnet2_promedio = cp.mean(self.mag_vals[:, Neq:]**2, axis=1)
      magnet_abs_promedio = cp.mean(cp.abs(self.mag_vals[:, Neq:]), axis=1)

      # Cv, Xs y Xs_abs
      cv = (energia2_promedio - energia_promedio**2) / (temperaturas**2)/self.num_spin
      X_s = (magnet2_promedio - magnet_promedio**2) /temperaturas/self.num_spin
      X_sabs = (magnet2_promedio - magnet_abs_promedio**2) /temperaturas/self.num_spin
      #magnetizacion = cp.sum(self.espines_blancos) +  cp.sum(self.espines_negros)

      # Convertir resultados a CPU
      self.cv_cpu = cp.asnumpy(cv)
      self.X_s_cpu = cp.asnumpy(X_s)
      self.X_sabs_cpu = cp.asnumpy(X_sabs)
      self.temperaturas_cpu = cp.asnumpy(temperaturas)
      self.magnetizacion_tot =cp.asnumpy(magnet_promedio)

  def energia_total(self):
    # Alcanza con sumar sobre los blancos y sus vecinos
    energia = -self.J * cp.sum(self.espines_blancos * cp.sum(self.espines_negros[self.vecinos_blancos[:, :, 1]], axis=1))
    return energia

  def magnetizacion_total(self):
    magnetizacion = cp.sum(self.espines_blancos) +  cp.sum(self.espines_negros)
    return magnetizacion

  def plot_configuration(self):
    """Genera una matriz LxL de la configuración actual de espines y la grafica"""
    configuracion = np.zeros((self.L, self.L), dtype=int)

    indice_blanco = 0
    indice_negro = 0
    for i in range(self.L):
        for j in range(self.L):
            if (i + j) % 2 == 0:
                configuracion[i, j] = cp.asnumpy(self.espines_blancos[indice_blanco])
                indice_blanco += 1
            else:
                configuracion[i, j] = cp.asnumpy(self.espines_negros[indice_negro])
                indice_negro += 1

    # Visualización de la configuración usando imshow
    plt.figure(figsize=(6, 6))
    plt.imshow(configuracion, cmap='bwr', vmin=-1, vmax=1)
    plt.colorbar(label="Espín")
    plt.title(f"Configuración de Espines a T={self.T_inicial}, L={self.L}")
    plt.xlabel("Posición X")
    plt.ylabel("Posición Y")
    plt.show()




# Implementación de clase:

El siguiente código se utilizó para tomar mediciones en la region de temperaturas entre 0 y 5, guardando información de cada paso y tambien guardando información de parámetros termodinámicos para cada temperatura.

En la siguiente sección se muestra el codigo utilizado para observar los datos a partir del archivo de salida.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Primer segmento: 10 puntos entre 0 y 2.15
segmento1 = np.linspace(0, 2.15, 10, endpoint=False)
# Segundo segmento: 100 puntos entre 2.15 y 2.55
segmento2 = np.linspace(2.15, 2.55, 100, endpoint=False)
# Tercer segmento: 10 puntos entre 2.55 y 5
segmento3 = np.linspace(2.55, 5, 10)
# Combinar todos los segmentos en un solo array
Temps = np.concatenate([segmento1, segmento2, segmento3])

# Parámetros de simulación
L =  # Tamaño de red
tag = #Identificador para el visor

steps_mc = 50000                  # Pasos de Monte Carlo
Neq = int(0.5 * steps_mc)         # Pasos de equilibración

# Inicializar el modelo
modelo = Ising2D(Temps, L=L, random_init=False)

# Simular para el rango de temperaturas
modelo.simu( steps_mc=steps_mc, Neq=Neq)


resultados = {
    "T": [],                        # Temperatura
    "paso_MC": [],                  # Paso de Monte Carlo
    "energia": [],                  # Energía por paso
    "magnetizacion": [],            # Magnetización por paso
}

# Guardar también los promedios y propiedades térmicas para cada temperatura
resumen = {
    "T": modelo.temperaturas_cpu.tolist(),
    "susceptibilidad": modelo.X_s_cpu.tolist(),
    "susceptibilidad_abs": modelo.X_sabs_cpu.tolist(),
    "calor_especifico": modelo.cv_cpu.tolist(),
    "Magnetizacion_tot": modelo.magnetizacion_tot.tolist(),
}

# Llenar el contenedor con los valores paso a paso
for idx, T in enumerate(modelo.temperaturas_cpu):
    for paso in range(steps_mc):
        resultados["T"].append(T)
        resultados["paso_MC"].append(paso)
        resultados["energia"].append(modelo.energia_vals[idx, paso].get())  # Convertir de CuPy a NumPy
        resultados["magnetizacion"].append(modelo.mag_vals[idx, paso].get())  # Convertir de CuPy a NumPy

# Convertir a DataFrame
resultados_df = pd.DataFrame(resultados)
resumen_df = pd.DataFrame(resumen)

# Guardar resultados paso a paso en un archivo CSV
filename_datos = f"resultados_paso_a_paso_L{tag}_{L}.csv"
resultados_df.to_csv(filename_datos, index=False)
print(f"Resultados paso a paso guardados en '{filename_datos}'")

# Guardar el resumen de propiedades térmicas en otro archivo CSV
filename_resumen = f"resumen_propiedades_L{tag}_{L}.csv"
resumen_df.to_csv(filename_resumen, index=False)
print(f"Resumen de propiedades guardado en '{filename_resumen}'")


# Visor de archivos y graficadora.

La siguiente celda, se usó para ver parametros en funcion de los pasos de MC o el promedio para cada temperatura.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, widgets
import os

# Definimos las temperaturas personalizadas
# Primer segmento: 10 puntos entre 0 y 2.15
segmento1 = np.linspace(0, 2.15, 10, endpoint=False)
# Segundo segmento: 100 puntos entre 2.15 y 2.55
segmento2 = np.linspace(2.15, 2.55, 100, endpoint=False)
# Tercer segmento: 10 puntos entre 2.55 y 5
segmento3 = np.linspace(2.55, 5, 10)
# Combinar todos los segmentos en un solo array
Temps = np.concatenate([segmento1, segmento2, segmento3])

# Función para cargar un archivo basado en el tamaño L
def cargar_archivo(L):
    filename_datos = f"resultados_paso_a_paso_L_{L}.csv"
    filename_resumen = f"resumen_propiedades_L_{L}.csv"

    if os.path.exists(filename_datos) and os.path.exists(filename_resumen):
        resultados_df = pd.read_csv(filename_datos)
        resumen_df = pd.read_csv(filename_resumen)
        print(f"Archivos para L={L} cargados correctamente.")
        return resultados_df, resumen_df
    else:
        print(f"Archivos para L={L} no encontrados.")
        return None, None


# Función para graficar paso a paso
def graficar_paso_a_paso(resultados_df, temperatura_input, variable, paso_inicial, exportar=False, L=10):
    # Seleccionar la temperatura más cercana
    temperatura = Temps[np.abs(Temps - temperatura_input).argmin()]
    print(f"Temperatura seleccionada más cercana: {temperatura}")
    
    datos_temp = resultados_df[resultados_df["T"] == temperatura]
    datos_temp = datos_temp[datos_temp["paso_MC"] >= paso_inicial]
    
    if variable == "Energía":
        plt.plot(datos_temp["paso_MC"], datos_temp["energia"]/L**2, label=f"Energía (T={temperatura})")
        plt.ylabel("Energía")
    elif variable == "Magnetización":
        plt.plot(datos_temp["paso_MC"], datos_temp["magnetizacion"]/L**2, label=f"Magnetización (T={temperatura})")
        plt.ylabel("Magnetización")
    
    plt.xlabel("Paso de Monte Carlo")
    plt.title(f"{variable} paso a paso para T={temperatura}")
    plt.legend()
    plt.grid()
    plt.show()

    # Exportar datos si se seleccionó
    if exportar:
        filename = f"datos_paso_a_paso_T_{temperatura}.csv"
        datos_temp.to_csv(filename, index=False)
        print(f"Datos exportados a {filename}")


# Función para graficar propiedades promedio
def graficar_propiedades(resumen_df, variable, exportar=False,L=10):
    if variable == "Susceptibilidad":
        plt.plot(resumen_df["T"], resumen_df["susceptibilidad"], label="Susceptibilidad")
        plt.ylabel("Susceptibilidad")
    elif variable == "Susceptibilidad Abs":
        plt.plot(resumen_df["T"], resumen_df["susceptibilidad_abs"], label="Susceptibilidad Abs")
        plt.ylabel("Susceptibilidad Abs")
    elif variable == "Calor Específico":
        plt.plot(resumen_df["T"], resumen_df["calor_especifico"], label="Calor Específico")
        plt.ylabel("Calor Específico")
    elif variable == "Magnetización Total":
        plt.plot(resumen_df["T"], resumen_df["Magnetizacion_tot"]/L**2, label="Magnetización Total")
        plt.ylabel("Magnetización Total")
    elif variable == "Magnetización Total Abs":
        resumen_df["Magnetizacion_tot_abs"] = resumen_df["Magnetizacion_tot"].abs()
        plt.plot(resumen_df["T"], resumen_df["Magnetizacion_tot_abs"], label="Magnetización Total Abs")
        plt.ylabel("Magnetización Total Abs")

    plt.xlabel("Temperatura")
    plt.title(f"{variable} en función de la temperatura")
    plt.legend()
    plt.grid()
    plt.show()

    # Exportar datos si se seleccionó
    if exportar:
        filename = f"resumen_{variable.replace(' ', '_')}.csv"
        resumen_df.to_csv(filename, index=False)
        print(f"Datos exportados a {filename}")


# Función principal interactiva
def menu(L, tipo_datos, temperatura_input=None, variable=None, paso_inicial=0, exportar=False):
    resultados_df, resumen_df = cargar_archivo(L)
    if resultados_df is None or resumen_df is None:
        return
    
    if tipo_datos == "Paso a Paso":
        if temperatura_input is None or variable is None:
            print("Seleccione una temperatura y una variable para graficar.")
        else:
            graficar_paso_a_paso(resultados_df, temperatura_input, variable, paso_inicial, exportar,L)
    elif tipo_datos == "Propiedades Promedio":
        if variable is None:
            print("Seleccione una variable para graficar.")
        else:
            graficar_propiedades(resumen_df, variable, exportar,L)


# Crear widgets interactivos
L_widget = widgets.IntText(value=10, description="L:")
tipo_datos_widget = widgets.RadioButtons(
    options=["Paso a Paso", "Propiedades Promedio"],
    description="Tipo de Datos:"
)
temperatura_widget = widgets.FloatText(value=2.15, description="Temperatura Input:")
variable_widget = widgets.Dropdown(
    options=["Energía", "Magnetización", "Susceptibilidad", "Susceptibilidad Abs", "Calor Específico", "Magnetización Total", "Magnetización Total Abs"],
    description="Variable:"
)
paso_inicial_widget = widgets.IntText(value=0, description="Paso Inicial:")
exportar_widget = widgets.Checkbox(value=False, description="Exportar Datos")

# Conectar widgets al menú
interact(
    menu,
    L=L_widget,
    tipo_datos=tipo_datos_widget,
    temperatura_input=temperatura_widget,
    variable=variable_widget,
    paso_inicial=paso_inicial_widget,
    exportar=exportar_widget,
)


El siguiente se utilizó para graficar la energia libre; En este caso se utilizó un tag, para identificar a las distintas simulaciones de igual L (mas que nada para mismo L pero distinta cantidad de pasos de MC).
Tiene una opción para suavizar datos pero no la recomiendo mcuho ya que se puede perder información.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
from scipy.ndimage import gaussian_filter1d
import os

# Función para cargar un archivo basado en el tamaño L
def cargar_archivo(L,tag):
    filename_datos = f"resultados_paso_a_paso_L{tag}_{L}.csv"

    if os.path.exists(filename_datos) :
        resultados_df = pd.read_csv(filename_datos)
        print(f"Archivos para L={L} cargados correctamente.")
        return resultados_df
    else:
        print(f"Archivos para L={L} no encontrados.")
        return None, None

# Función para calcular la energía libre a partir del histograma
def calcular_energia_libre(resultados_df, temperatura_input, L, bins=50, sigma_suavizado=1.0, exportar=False, ylim=50):
    Temps = resultados_df["T"].unique()  # Temperaturas disponibles en los datos
    temperatura = Temps[np.abs(Temps - temperatura_input).argmin()]  # Más cercana
    tolerancia = 1e-3  # Tolerancia para seleccionar datos
    datos_temp = resultados_df[np.abs(resultados_df["T"] - temperatura) < tolerancia]

    # Verificar si hay datos para la temperatura seleccionada
    if datos_temp.empty:
        print(f"No hay datos para T={temperatura} dentro de la tolerancia {tolerancia}.")
        return
    #neq = int(len(datos_temp["magnetizacion"])/2)
    # Extraer datos de magnetización
    magnetizacion = datos_temp["magnetizacion"]
    hist, bin_edges = np.histogram(magnetizacion, bins=bins, density=True)
    
    # Suavizar el histograma
    hist_suavizado = gaussian_filter1d(hist, sigma=sigma_suavizado)
    probabilidad = hist_suavizado / np.sum(hist_suavizado)  # Normalizar nuevamente

    # Valores centrales de los bins
    bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

    # Energía libre: F(m, T) = -T * log(P(m, T))
    epsilon = 1e-5  # Evitar log(0)
    energia_libre = -temperatura * np.log(probabilidad + epsilon)

    # Normalizar la energía libre
    energia_libre -= np.min(energia_libre)

    # Gráfica
    plt.figure(figsize=(8, 6))
    plt.plot(bin_centers/L**2, energia_libre, marker='o',linestyle='none', label=f"T={temperatura}, L={L}")
    plt.xlabel("Magnetización (m)",fontsize=16)
    plt.ylabel("Energía Libre $F_L(m, T)$",fontsize=16)
    plt.ylim([-0.4,float(ylim)])
    #plt.xlim([-1.1,1.1])
    
    plt.title(f"Energía Libre para L={L} y T={temperatura}",fontsize=14)
    plt.grid()
    plt.legend(fontsize=14)
    plt.show()

    # Calcular barrera de energía
    ms_idx = np.argmax(probabilidad)  # Índice del valor más probable
    ms = bin_centers[ms_idx]
    F_0 = energia_libre[np.argmin(np.abs(bin_centers - 0))]  # F_L(0, T)
    F_ms = energia_libre[ms_idx]  # F_L(m_s, T)
    delta_F = F_0 - F_ms
    print(f"Barreras de Energía: ΔF = F_L(0, T) - F_L(±ms, T) = {delta_F:.3f}")

    # Exportar si se selecciona
    if exportar:
        filename = f"energia_libre_L_{L}_T_{temperatura:.2f}.csv"
        pd.DataFrame({"Magnetización": bin_centers, "Energía Libre": energia_libre}).to_csv(filename, index=False)
        print(f"Datos de energía libre exportados a {filename}")

# Función principal interactiva para la energía libre
def menu_energia_libre(L, temperatura_input, bins=50, sigma_suavizado=1.0, y_lim=50, tag='50k', exportar=False):
    resultados_df = cargar_archivo(L,tag)
    if resultados_df is None:
        return
    calcular_energia_libre(resultados_df, temperatura_input, L, bins, sigma_suavizado, exportar, y_lim)

# Widgets interactivos
L_widget = widgets.IntText(value=10,description="L:")
temperatura_widget = widgets.FloatText(value=2.1, description="Temperatura Input:")
bins_widget = widgets.IntSlider(value=50, min=10, max=200, step=10, description="Bins:")
sigma_widget = widgets.FloatSlider(value=1.0, min=0.01, max=5.0, step=0.1, description="Suavizado σ:")
y_lim_widget = widgets.IntText(value = 50, description="Límite superior plot.")
tag_wind = widgets.Text(value = '50k', descriptor = 'Tag para los datos:')
exportar_widget = widgets.Checkbox(value=False, description="Exportar Datos")

# Conectar widgets al menú
interact(
    menu_energia_libre,
    L=L_widget,
    temperatura_input=temperatura_widget,
    bins=bins_widget,
    sigma_suavizado=sigma_widget,
    y_lim = y_lim_widget,
    tag = tag_wind,
    exportar=exportar_widget,
)



Por último, la siguiente celda es para mostrar la correlación temporal; si bien no se incluyó este tipo de gráficas en el informe el analisis de estas, se usó como justificación para la explicación de muchos de los efectos de tamaño finito.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from ipywidgets import interact, widgets
import os

# Crear las temperaturas personalizadas
segmento1 = np.linspace(0, 2.15, 10, endpoint=False)
segmento2 = np.linspace(2.15, 2.55, 100, endpoint=False)
segmento3 = np.linspace(2.55, 5, 10)
Temps = np.concatenate([segmento1, segmento2, segmento3])

# Función para cargar un archivo basado en el tamaño L
def cargar_archivo(L):
    filename_datos = f"resultados_paso_a_paso_L_{L}.csv"
    if os.path.exists(filename_datos):
        resultados_df = pd.read_csv(filename_datos)
        print(f"Archivo para L={L} cargado correctamente.")
        return resultados_df
    else:
        print(f"Archivo para L={L} no encontrado.")
        return None

# Función de autocorrelación
def autocorrelacion(data, t_fin):
    n = len(data)  # Número total de datos
    mean_data = np.mean(data)  # <A>
    mean_data_sq = mean_data ** 2  # <A>^2
    mean_data2 = np.mean(data ** 2)  # <A^2>

    # Inicializa el array de autocorrelación
    correlacion = np.zeros(t_fin)

    # Calcula C_A(t) para cada t (desplazamiento de tiempo)
    for t in range(t_fin):
        productos = [
            data[t0] * data[t0 + t] for t0 in range(n - t)  # A(t + t_0) A(t_0)
        ]
        mean_productos = np.mean(productos)  # Promedio de los productos
        correlacion[t] = (mean_productos - mean_data_sq) / (mean_data2 - mean_data_sq)

    return correlacion

# Función para estimar el límite superior del ajuste
def estimar_limite_superior(C, umbral=0.1):
    max_valor = C[0]  # Valor inicial de la autocorrelación
    for i in range(len(C)):
        if C[i] < umbral * max_valor:  # Si cae por debajo del umbral relativo
            return i
    return len(C)  # Si no hay punto por debajo del umbral, tomar todo

# Definimos la función exponencial para el ajuste
def exponencial(t, tau):
    return np.exp(-t / tau)

# Función para calcular tau ajustando la exponencial
def calcular_tau(datos, max_t):
    t_values = np.arange(max_t)
    correlacion_corta = datos[:max_t]

    # Ajuste de la exponencial
    popt, _ = curve_fit(exponencial, t_values, correlacion_corta)
    tau = popt[0]  # Tau ajustado
    return tau

# Función para calcular y graficar la autocorrelación
def calcular_autocorrelacion(resultados_df, temperatura_input, variable, t_fin, umbral=0.1):
    # Seleccionar la temperatura más cercana
    temperatura = Temps[np.abs(Temps - temperatura_input).argmin()]
    print(f"Temperatura seleccionada más cercana: {temperatura}")

    # Filtrar los datos para la temperatura seleccionada
    datos_temp = resultados_df[resultados_df["T"] == temperatura]

    # Seleccionar la variable para calcular la autocorrelación
    if variable == "Energía":
        data = datos_temp["energia"].to_numpy()
    elif variable == "Magnetización":
        data = datos_temp["magnetizacion"].to_numpy()
    elif variable == "Magnetización Abs":
        data = np.abs(datos_temp["magnetizacion"].to_numpy())
    else:
        raise ValueError(f"Variable desconocida: {variable}")

    # Descartar la primera mitad de los datos (termalización)
    Neq = int(len(data) / 2)
    data = data[Neq:]

    # Calcular la autocorrelación
    correlacion = autocorrelacion(data, t_fin)

    # Estimar límite superior para el ajuste
    max_t = estimar_limite_superior(correlacion, umbral=umbral)
    print(f"Límite superior estimado para el ajuste: {max_t}")

    # Calcular tau
    tau = calcular_tau(correlacion, max_t)
    print(f"Tiempo de correlación (tau): {tau:.2f}")

    # Graficar la autocorrelación
    plt.figure(figsize=(8, 6))
    plt.plot(correlacion, marker='o', linestyle='-', label="Autocorrelación")
    plt.axvline(x=max_t, color='r', linestyle='--', label="Límite Superior Ajuste")
    plt.xlabel("Desplazamiento de tiempo (t)")
    plt.ylabel("C_A(t)")
    plt.title(f"Autocorrelación de {variable} a T={temperatura}")
    plt.legend()
    plt.grid()
    plt.show()

# Función principal interactiva
def menu_autocorrelacion(L, temperatura_input, variable, t_fin, umbral):
    resultados_df = cargar_archivo(L)
    if resultados_df is None:
        return
    calcular_autocorrelacion(resultados_df, temperatura_input, variable, t_fin, umbral)

# Widgets interactivos
L_widget = widgets.IntText(value=10, description="L:")
temperatura_widget = widgets.FloatText(value=2.1, description="Temperatura Input:")
t_f_widget = widgets.IntText(value=500, description="Limite superior calculo:")
variable_widget = widgets.Dropdown(
    options=["Energía", "Magnetización", "Magnetización Abs"],
    description="Variable:",
)
umbral_widget = widgets.FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description="Umbral Ajuste:")

# Conectar widgets al menú
interact(
    menu_autocorrelacion,
    L=L_widget,
    temperatura_input=temperatura_widget,
    t_fin=t_f_widget,
    variable=variable_widget,
    umbral=umbral_widget,
)
